In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io, filters
import numpy as np

from tqdm.notebook import tqdm

from PIL import Image
import numpy as np

import cv2
import copy
import time

from mpire import WorkerPool

from PIL import Image
from matplotlib import cm
from tqdm.notebook import tqdm


from combra import stats as cstats
from combra import approx as capprox
from combra import image as cimage

from numba import njit
import numpy as np
import cv2
import os

from combra.tests import test_fractal_dimensions
from combra.contours import contour_to_binary_mask, scale_contour, draw_contours



In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = img[400:-400,1300:-1300]
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)

    cnts, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    return binary, cnts


images_paths=[
            #   'data/images/0005.JPG',
              'data/images/0500.JPG',
              'data/images/1000.JPG',
              'data/images/1500.JPG',
              'data/images/2000.JPG',
              'data/images/2500.JPG',
              'data/images/3000.JPG',
              'data/images/3500.JPG',
              'data/images/4000.JPG',
              'data/images/4321.JPG',
              ]

images_names=[os.path.basename(image_path) for image_path in images_paths]



# Fractal dimention

In [ ]:
sizes = 2 ** np.arange(1, 10)
test_fractal_dimensions(sizes)

In [ ]:
types=['fractal_dimension']
type=types[0]
n_jobs=20

# step=1
n_jobs=20

for step in tqdm([0.01, 0.05,0.1,0.2,0.3]):

    legend=[]

    fig, axes= plt.subplots(2,4,figsize=(40,14))
    
    # Define 10 distinct colors
    colors = plt.cm.tab10(np.linspace(0, 1, 10))
    
    n_list=[]
    p_list=[]
    lambda_list=[]
    mu_list=[]
    sigma_list=[]
    a_list=[]

    for idx, image_path in enumerate(images_paths):

        binary, contours = preprocess_image(image_path)

        # Process in parallel - NO shared_objects
        with WorkerPool(n_jobs=n_jobs, use_dill=False) as pool:
            results = pool.map(
                cimage.contour_fractal_dimension,
                contours,
                progress_bar=True,
                chunk_size=8
            )

        len_list = [d for d in results if d is not None]


        x_orig, y_orig = cstats.stats_preprocess(len_list, step)


        # y_orig = np.log(y_orig)
        
        # Calculate mode (x value with maximum y)
        mode_idx = np.argmax(y_orig)
        mode = x_orig[mode_idx]
        
        # Get color for this image (cycle through colors if more than 10 images)
        color = colors[idx % len(colors)]

        # Fit binomial distribution
        # (x_fit, y_fit), n, p, amp = capprox.binomial_approx(x_orig, y_orig, n=25,p=0.2, x_lim=[0,5], N=100)
        # axes[0,0].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Poisson distribution
        # (x_fit, y_fit), lam, amp_poisson = capprox.poisson_approx(x_orig, y_orig, x_lim=[-5,5], N=100)
        # axes[0,1].plot(x_fit, y_fit, ':', linewidth=2, color=color)

        # Fit Gaussian distribution
        # (x_fit, y_fit), mu, sigma, amp_gauss = capprox.gauss_approx(x_orig, y_orig, mu=3, sigma=3, amp=1, x_lim=[0,5], N=100)
        # axes[0,2].plot(x_fit, y_fit, '-.', linewidth=2, color=color)

        # Fit Exponential distribution
        (x_fit, y_fit), a, amp_exp = capprox.exponential_approx(x_orig, y_orig, a=1, amp=1, x_lim=[1,2], N=20)
        axes[0,3].plot(x_fit, y_fit, '--', linewidth=2, color=color)

        axes[0, 0].set_title(f'binomial fit, step={step}',fontsize=15)
        axes[0,1].set_title(f'poisson fit, step={step}',fontsize=15)
        axes[0,2].set_title(f'gaussian fit, step={step}',fontsize=15)
        axes[0,3].set_title(f'exponential fit, step={step}',fontsize=15)

        n=0
        p=0
        mu=0
        sigma=0
        lam=0
        
        n_list.append(n)
        p_list.append(p)
        lambda_list.append(lam)
        mu_list.append(mu)
        sigma_list.append(sigma)
        a_list.append(a)

        # Plot original data with the same color (include mode in legend)
        axes[0,0].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, n={n:.0f}, p={p:.3f}')
        axes[0,1].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, λ={lam:.2f}' )
        axes[0,2].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, μ={mu:.2f}, σ={sigma:.2f}' )
        axes[0,3].plot(x_orig, y_orig, '-o', color=color, label=f'{image_path.split("/")[-1][:-4]}, m={mode:.0f}, a={a:.4f}' )
    
    
    axes[1,0].set_title(f'binomial fit, step={step}',fontsize=15)
    axes[1,1].set_title(f'poisson fit, step={step}',fontsize=15)
    axes[1,2].set_title(f'gaussian fit, step={step}',fontsize=15)
    axes[1,3].set_title(f'exponential fit, step={step}',fontsize=15)

    # Use dual y-axis for n and p (n on left, p on right)
    ax1 = axes[1,0]  # Left axis for n
    ax1.plot(range(len(images_paths)), n_list, '-o', label='n', color='blue')
    ax1.set_ylabel('n', color='blue', fontsize=15)
    ax1.tick_params(axis='y', labelcolor='blue', labelsize=15)
    # axes[1,0].set_yscale('log')
    
    # Create second y-axis on the right for p
    ax2 = ax1.twinx()  # Right axis for p
    ax2.plot(range(len(images_paths)), p_list, '-o', label='p', color='red')
    ax2.set_ylabel('p', color='red', fontsize=15)
    ax2.tick_params(axis='y', labelcolor='red', labelsize=15)
    axes[1,0].twinx().set_yscale('log')
    
    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=15)
    
    axes[1,1].plot(range(len(images_paths)), lambda_list, '-o', label='λ')
    axes[1,2].plot(range(len(images_paths)), mu_list, '-o', label='mu')
    axes[1,2].plot(range(len(images_paths)), sigma_list, '-o', label='sigma')
    axes[1,3].plot(range(len(images_paths)), a_list, '-o', label='a')

    for ax in [axes[1,0], axes[1,1], axes[1,2], axes[1,3]]:
        ax.set_xticks(range(len(images_paths)))
        ax.set_xticklabels(images_names, rotation=45, fontsize=15, ha='right')

    axes[0,0].legend( loc='upper right',fontsize=15)
    axes[0,1].legend( loc='upper right',fontsize=15)
    axes[0,2].legend( loc='upper right',fontsize=15)
    axes[0,3].legend( loc='upper right',fontsize=15)
    axes[1,1].legend( loc='upper right',fontsize=15)
    axes[1,2].legend( loc='upper right',fontsize=15)
    axes[1,3].legend( loc='upper right',fontsize=15)

    # axes[0,0].set_ylim(0,0.2)
    # axes[0,1].set_ylim(0,0.2)
    # axes[0,2].set_ylim(0,0.2)
    

    axes[0,0].set_xlim(1,2)
    axes[0,1].set_xlim(1,2)
    axes[0,2].set_xlim(1,2)
    axes[0,3].set_xlim(1,2)

    axes[0,0].set_yscale('log')
    axes[0,1].set_yscale('log')
    axes[0,2].set_yscale('log')
    axes[0,3].set_yscale('log')

    axes[0,3].set_ylim(1e-8, 1e6)


    axes[0,0].set_ylabel('p(x)',fontsize=15)
    axes[0,1].set_ylabel('p(x)',fontsize=15)
    axes[0,2].set_ylabel('p(x)',fontsize=15)
    axes[0,3].set_ylabel('p(x)',fontsize=15)

    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    # plt.savefig(f'countour_areas_step={step}.jpg',bbox_inches='tight')

    plt.show()

In [ ]:
y_orig